In [46]:
import os
os.chdir('test_split')
import geojson
from dateutil.parser import parse
from glob import glob

FileNotFoundError: [Errno 2] No such file or directory: 'test_split'

In [72]:
def split_geojson(input_geojson, folder_name, split_minutes=5):
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
    
    with open(input_geojson, 'r') as f:
        input_points = geojson.load(f)
        
    basename = os.path.join(folder_name, os.path.basename(input_geojson).replace(".geojson", ""))
        
    write_first_point(input_points, basename, split_minutes)
    write_lines(input_points, basename, split_minutes)
    write_interval(input_points, basename, interval=10, split_minutes=split_minutes, name='10min')
    write_interval(input_points, basename, interval=60, split_minutes=split_minutes, name='Hourly')

In [48]:
def write_first_point(input_points, basename, split_minutes):
    first_point = geojson.FeatureCollection([input_points[0]])
    time = parse(input_points[0].properties['time'])
    time_str = f"{time.hour:02}{(time.minute // split_minutes) * split_minutes:02}"
    output_filename = basename + f"_FirstPoint_{time_str}.geojson"
    with open(output_filename, 'w') as f:
        geojson.dump(first_point, f)

In [49]:
def feature_from_line_coords(segment):
    line_string = geojson.LineString(coordinates=segment)
    feature = geojson.Feature(geometry=line_string)
    
    return feature

In [50]:
def write_lines(input_points, basename, split_minutes):
    line_segments = {}
    current_line_seg = []

    # Split the lines
    for feature in input_points.features:
        coords = feature.geometry.coordinates
        time = parse(feature.properties['time'])

        current_line_seg.append(coords)
        if time.minute % split_minutes == 0:
            if len(current_line_seg) > 1:
                # If we've got something from before, then add to list of segs
                time_str = f"{time.hour:02}{(time.minute // split_minutes) * split_minutes:02}"
                line_segments[time_str] = current_line_seg
            # Start new line segment
            current_line_seg = [coords]

    # Deal with any left over at the end
    time_str = f"{time.hour:02}{((time.minute // split_minutes) + 1) * split_minutes:02}"
    line_segments[time_str] = current_line_seg

    # Write them out to separate files
    for time, segment in line_segments.items():
        feature = feature_from_line_coords(segment)
        feature_collection = geojson.FeatureCollection([feature])
        with open(basename + '_Lines_' + str(time) + '.geojson', 'w') as f:
            geojson.dump(feature_collection, f)
    

In [51]:
def write_interval(input_points, basename, interval, split_minutes, name):
    points = {}
    current_points = []

    for feature in input_points.features:
        time = parse(feature.properties['time'])

        if time.minute % interval == 0:
            properties = feature.properties
            time = parse(properties['time'])
            properties['time_str'] = f"{time.hour:02}{time.minute:02}"
            current_points.append(geojson.Feature(geometry=feature.geometry, properties=properties))
            
        if time.minute % split_minutes == 0:
            if len(current_points) > 0:
                time_str = f"{time.hour:02}{(time.minute // split_minutes) * split_minutes:02}"
                points[time_str] = current_points
            current_points = []


    for time, feature in points.items():
        feature_collection = geojson.FeatureCollection(feature)
        with open(basename + '_' + name + "_" + str(time) + '.geojson', 'w') as f:
            geojson.dump(feature_collection, f)

In [52]:
def generate_layer_list(time_strs):
    layer_list = ""
    
    layer_template = """<Layer id="1995-12-12 {time}" name="1995-12-12 {time}">
    <Layer id="Nelson 1995-12-12 {time}" name="Nelson"/>
    <Layer id="Collingwood 1995-12-12 {time}" name="Collingwood"/>
</Layer>"""
    
    
    for time_str in time_strs:
        layer_list += "\n" + layer_template.format(time=time_str)
        
    return layer_list

In [53]:
def generate_content_layer(time_str):
    time_no_colon = time_str.replace(":", "")
    content = ""
    
    content += f'<IfLayerOn layerId="1995-12-12 {time_str}">\n'
    
    content += generate_ship_content_layer(time_str, "Nelson") + '\n\n'
    
    content += generate_ship_content_layer(time_str, "Collingwood")
    
    content += "</IfLayerOn>"
    
    return content

In [54]:
def generate_ship_content_layer(time_str, ship_name):
    time_no_colon = time_str.replace(":", "")
    
    color = SHIP_NAME_TO_COLOR[ship_name]
    content = ""
    
    
    content += '    <IfLayerOn layerId="{ship_name} 1995-12-12 {time}">\n'
    
    if os.path.exists(f"{ship_name}_Lines_{time_no_colon}.geojson"):
        content += """        <Vector dataset="{ship_name}_Lines_{time_no_colon}.geojson" layer="{ship_name}_Lines_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='PEN(c:{color},w:5px)'/>\n"""
    
    if os.path.exists(f"{ship_name}_10min_{time_no_colon}.geojson"):
        content += """        <Vector dataset="{ship_name}_10min_{time_no_colon}.geojson" layer="{ship_name}_10min_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='SYMBOL(c:{color},s:2,id:"ogr-sym-3")'>
                <LogicalStructure displayLayerName="{ship_name}" fieldToDisplay="time"/>
        </Vector>\n"""
        
    if os.path.exists(f"{ship_name}_Hourly_{time_no_colon}.geojson"):
        content += """        <VectorLabel dataset="{ship_name}_Hourly_{time_no_colon}.geojson" layer="{ship_name}_Hourly_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='LABEL(t:{{time_str}},c:{color},s:24pt,p:4,dx:7mm,bo:1)'/>\n"""
    
    if os.path.exists(f"{ship_name}_FirstPoint_{time_no_colon}.geojson"):
        content += """        <VectorLabel dataset="{ship_name}_FirstPoint_{time_no_colon}.geojson" layer="{ship_name}_FirstPoint_{time_no_colon}" georeferencingId="georeferenced" ogrStyleString='LABEL(t:"{ship_name}",c:{color},s:24pt,p:2,dy:10mm,bo:1)'/>\n"""
    
    content += "    </IfLayerOn>"
    
    return content.format(time=time_str, time_no_colon=time_no_colon, ship_name=ship_name, color=color)

In [63]:
def generate_js_timestamps(time_strs):
    content = "var timestamps = [\n"
    
    template = """{{
    name: &quot;1995-12-12 {time_str}&quot;,
    ocg_name: &quot;1995-12-12 {time_str}&quot;
}},\n"""
    
    for time_str in time_strs:
        content += template.format(time_str=time_str)
    
    content += "];\n"
    
    return content

In [41]:
SHIP_NAME_TO_COLOR = {'Nelson': '#bd1b44',
                      'Collingwood': '#0d0d8e'}

In [75]:
def create_xml(output_filename, folder_name):
    os.chdir(folder_name)
    files = glob("*_*_*.geojson")
    times_set = {os.path.splitext(filename)[0].split("_")[2] for filename in files}
    time_strs = [f"{time[:2]}:{time[2:]}" for time in sorted(times_set)]

    layer_list = generate_layer_list(time_strs)

    content_layer_strs = [generate_content_layer(time_str) for time_str in time_strs]
    whole_content_layer = "\n\n".join(content_layer_strs)

    js_timestamps = generate_js_timestamps(time_strs)
    
    with open('../template.xml') as f:
        template = f.read()
    
    template = template.replace("!!CONTENT", whole_content_layer)
    template = template.replace("!!JS_TIMESTAMPS", js_timestamps)
    template = template.replace("!!LAYER_LIST", layer_list)

    with open(output_filename, 'w') as f:
        f.write(template)
    os.chdir('..')

In [81]:
os.chdir('..')

In [103]:
minutes = [30, 15, 10, 5, 3, 2, 1]

for minute in minutes:
    split_geojson("Nelson.geojson", f'split_{minute}min', split_minutes=minute)
    split_geojson("Collingwood.geojson", f'split_{minute}min', split_minutes=minute)
    create_xml('output.xml', f'split_{minute}min')
    
    os.chdir(f'split_{minute}min')
    result = subprocess.run(f"python ../gdal_create_pdf.py output.xml split_{minute}min.pdf", shell=True, stderr=subprocess.PIPE).stderr
    if result == '':
        print("Created")
    os.chdir("..")



In [85]:
os.chdir('split_30min')

In [86]:
os.system(f"python gdal_create_pdf.py output.xml split_30min.pdf ")

512

In [87]:
import subprocess

In [97]:
subprocess.run(f"python ../gdal_create_pdf.py output.xml split_30min.pdf", shell=True, stderr=subprocess.PIPE).stderr

b''

In [99]:
os.chdir('..')

In [101]:
os.getcwd()

'/Users/robin/.Trash'

In [102]:
os.chdir('/Users/robin/Documents/IanMayo/GeoPDF/test_split')